# Example 3: Maximization of a multivariate function 

In this notebook the framework is used to optimize a function which depends on two variables $x_0$ and $x_1$. We illustrate both closed-loop and iterative approaches to solving the optimization problem using our framework. 

Similar to Example 2, we use here a known function to illustrate how the framework is applied, but it is not a requirement for the framework that the function can be written down explicitly as we do below. Hence, this option of using the framework allows for optimizing _unknown_ functions or _functions without explicit definitions_ and will come in handy for e.g. optimizing complex experiments. 

In the following we will maximize the Easom function, a classical optimization test function
$$
f(\mathbf{x}) = - \cos{(x_0)} \, \cos{(x_1)} \, \mathrm{e}^{-(x_0 - x_0^*)^2 - (x_1 - x_1^*)^2}, \quad \mathbf{x} = [x_0, x_1].
$$

It is known that the function above has its maximum at $\mathbf{x}^* = (x_0^*, x_1^*)$ (typical values used for the minimum are $\mathbf{x}^* = (\pi, \pi)$ but we will use $\mathbf{x}^* = (0,0)$). For more on the Easom function, see this page [https://www.sfu.ca/~ssurjano/easom.html](https://www.sfu.ca/~ssurjano/easom.html).


## Framework approach

We will solve the problem in three different ways:
1. using the closed-loop approach of the `.auto`-method
2. using the iterative optimization approach of the framework which requires using the methods `.ask` and `.tell`. This approach allows for iterative optimization and optimization of any callable function, known or otherwise.
3. extending the problem to set $x_1$ to be an integer, and again trying to find the maximum. In this case, it should be harder to find the global maximum because it will come at a single value of $x_1$. We solve this problem using the `.auto`-method.

The optimization process can be stopped after any number of iterations.


## Technical note

Installation of `torch` and `torchvision` (required dependencies) cannot be bundled as part of the `creative_project` installable. This is unfortunate, but a known issue it seems. Therefore these must be installed first, before installing `creative_project`.

### Get started: Import libraries

In [ ]:
# Preamble

# Install torch and torchvision. Use this link to identify the right versions to install on your system 
# depending on configuration: https://pytorch.org/get-started/locally/
#
#pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
#
# Install creative_project from github (will require authentication with password)
#pip install --user https://github.com/svedel/kre8_core/
! pip install --user greattunes

In [ ]:
# import
import pandas as pd
import numpy as np
import torch
from greattunes import TuneSession
import matplotlib.pyplot as plt
%matplotlib inline

### Setup the problem to be solved: define response function

Define the problem to optimize. Here we define a known function and use both the closed-loop solution method based on the `.auto`-method which requires the function to be specified as well as the iterative approach using the `.ask` and `.tell` methods that can be applied to both specified and unspecified, but sample-able functions that cannot be written down.

In [ ]:
# define the function (negative of the Easom function)
def neg_Easom(x):
    covar0, covar1 = np.meshgrid(x["covar0"].array, x["covar1"].array)
    return np.cos(covar0) * np.cos(covar1) * np.exp(-(covar0 ** 2 + covar1 ** 2))

Defines the ranges of the problem to be solved, and sets initial guess

In [ ]:
# limit on range
covar_lim = 5

# define the range of interest
x0_init = 1.0
x1_init = -1.0
covars2d = [(x0_init, -covar_lim, covar_lim), (x1_init, -covar_lim, covar_lim)]

Plots the response function

In [ ]:
# a plot-friendly version of the objective function
def plot_neg_Easom(x_vec):
    xp, yp = np.meshgrid(x_vec, x_vec)
    output = neg_Easom(pd.DataFrame({"covar0": x_vec, "covar1": x_vec}))
    return xp, yp, output

# generate the data for the figure
x = np.linspace(-covar_lim,covar_lim,200)
xp, yp, output = plot_neg_Easom(x)

# Set up a figure twice as tall as it is wide
fig = plt.figure(figsize=(12,6))  #plt.figaspect(2.)

# First subplot
ax = fig.add_subplot(1, 2, 1)
ax.contourf(xp, yp, output, cmap="jet")
ax.set_xlabel("covar_0")
ax.set_ylabel("covar_1")

# Second subplot
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(xp, yp, output, rstride=1, cstride=1,linewidth=1, antialiased=False, shade=False, cmap="jet")
ax.set_xlabel("covar_0")
ax.set_ylabel("covar_1")
ax.set_zlabel("-Easom(covar_0, covar_1)")

plt.show()

### Solution 1: Closed-loop solution approach using `.auto` method

Instantiate the `TuneSession` class and solve the problem

In [ ]:
# initialize class instance
cc = TuneSession(covars=covars2d)

# number of iterations
max_iter = 20

# run the auto-method
cc.auto(response_samp_func=neg_Easom, max_iter=max_iter)

Best guess after solving

In [ ]:
# run current_best method
cc.current_best()

### Solution 2: Iterative solution using `.ask` and `.tell` methods

Instantiate the `TuneSession` class and solve the problem. In this case, we need to write our own loop to iterate. Notice that the `covars` and `response` variables are converted to `torch` tensors of size $1 \times \mathrm{\#covariates}$ to store them in the instantiated class, where they are used for retraining the model at each iteration.

In [ ]:
from greattunes.data_format_mappings import tensor2pretty_covariate

# initialize the class instance
cc2 = TuneSession(covars=covars2d)

# run the solution
for i in range(max_iter):
    # generate candidate
    cc2.ask()

    # sample covariates and response
    # tensor2pretty_covariate maps between the backend dataformat used by 'proposed_X' to the pandas-based format consumed by the response function
    # the attribute 'covar_details' keeps information that maps backend and pandas ("pretty") dataformats
    covars = tensor2pretty_covariate(train_X_sample=cc2.proposed_X[-1].reshape(1,2), covar_details=cc2.covar_details)
    response = pd.DataFrame({"Response": [neg_Easom(covars)]})

    # report response
    cc2.tell(covar_obs=covars, response_obs=response)

Best guess after solving

In [ ]:
# run current_best method
cc2.current_best()

### Solution 3: Closed-loop solution via `.auto`, one of the covariates is an integer

In addition to the continuous covariates explored so far, the framework also handles integer and categorical covariates.

The framework determines the datatypes from the input provided in the tuple `covars` during class initialization, so it will assume a covariate is an integer if only integers are provided. Alternatively, `covars` can also be provided as a dict, which allows more control and transparency to users. We show how to use this in **Example XXX**

For now, we instantiate the `TuneSession` class while switching the second covariate to be an integer and solve the problem using the closed-loop `.auto`-method.

In [ ]:
# define the range of interest
x0_init = 1.0
x1_init = -1
covars2d_split = [(x0_init, -5, 5), (x1_init, -5, 5)]

In [ ]:
# plot the function
def plot_neg_Easom_split(x_vec):
    xint = np.round(x_vec,0)
    xp, yp = np.meshgrid(x_vec, xint)
    output = neg_Easom(pd.DataFrame({"covar0": x_vec, "covar1": xint}))
    return xp, yp, output

xp_split, yp_split, output_split = plot_neg_Easom_split(x)

# Set up a figure twice as tall as it is wide
fig = plt.figure(figsize=(12,6))  #plt.figaspect(2.)

# First subplot
ax = fig.add_subplot(1, 2, 1)
ax.contourf(xp_split, yp_split, output_split, cmap="jet")
ax.set_xlabel("covar_0 (continuous)")
ax.set_ylabel("covar_1 (integer)")

# Second subplot
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(xp_split, yp_split, output_split, rstride=1, cstride=1,linewidth=1, antialiased=False, shade=False, cmap="jet")
ax.set_xlabel("covar_0 (continuous)")
ax.set_ylabel("covar_1 (integer)")
ax.set_zlabel("-Easom(covar_0, covar_1)")

plt.show()

In [ ]:
# initialize class instance
cc3 = TuneSession(covars=covars2d_split)

# number of iterations
max_iter = 100

# run the auto-method
cc3.auto(response_samp_func=neg_Easom, max_iter=max_iter)

Best guess after solving

In [ ]:
# run current_best method
cc3.current_best()